# Basic model with static forcing


In [ ]:
# | include: false
import os

os.environ["USE_PYGEOS"] = "0"

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ribasim
from ribasim.config import Node
from ribasim.model import Model
from ribasim.nodes import (
    basin,
    discrete_control,
    flow_boundary,
    fractional_flow,
    level_boundary,
    level_demand,
    linear_resistance,
    manning_resistance,
    outlet,
    pid_control,
    pump,
    tabulated_rating_curve,
    user_demand,
)
from shapely.geometry import Point

In [ ]:
model = Model(starttime="2020-01-01 00:00:00", endtime="2021-01-01 00:00:00")

Setup the basins:


In [ ]:
time = pd.date_range(model.starttime, model.endtime)
day_of_year = time.day_of_year.to_numpy()
seconds_per_day = 24 * 60 * 60
evaporation = (
    (-1.0 * np.cos(day_of_year / 365.0 * 2 * np.pi) + 1.0) * 0.0025 / seconds_per_day
)
rng = np.random.default_rng(seed=0)
precipitation = (
    rng.lognormal(mean=-1.0, sigma=1.7, size=time.size) * 0.001 / seconds_per_day
)

# Convert steady forcing to m/s
# 2 mm/d precipitation, 1 mm/d evaporation

basin_data = [
    basin.Profile(area=[0.01, 1000.0], level=[0.0, 1.0]),
    basin.Time(
        time=pd.date_range(model.starttime, model.endtime),
        drainage=0.0,
        potential_evaporation=evaporation,
        infiltration=0.0,
        precipitation=precipitation,
        urban_runoff=0.0,
    ),
    basin.State(level=[1.4]),
]

model.basin.add(Node(1, Point(0.0, 0.0)), basin_data)
model.basin.add(Node(3, Point(2.0, 0.0)), basin_data)
model.basin.add(Node(6, Point(3.0, 2.0)), basin_data)
model.basin.add(Node(9, Point(5.0, 0.0)), basin_data)

Setup linear resistance:


In [ ]:
model.linear_resistance.add(
    Node(10, Point(6.0, 0.0)),
    [linear_resistance.Static(resistance=[5e3])],
)
model.linear_resistance.add(
    Node(12, Point(2.0, 1.0)),
    [linear_resistance.Static(resistance=[3600.0 * 24.0 / 100.0])],
)

Setup Manning resistance:


In [ ]:
model.manning_resistance.add(
    Node(2, Point(1.0, 0.0)),
    [
        manning_resistance.Static(
            length=[900], manning_n=[0.04], profile_width=[6.0], profile_slope=[3.0]
        )
    ],
)

Set up a rating curve node:


In [ ]:
model.tabulated_rating_curve.add(
    Node(4, Point(3.0, 0.0)),
    [tabulated_rating_curve.Static(level=[0.0, 1.0], flow_rate=[0.0, 10 / 86400])],
)

Setup fractional flows:


In [ ]:
model.fractional_flow.add(
    Node(5, Point(3.0, 1.0)), [fractional_flow.Static(fraction=[0.3])]
)
model.fractional_flow.add(
    Node(8, Point(4.0, 0.0)), [fractional_flow.Static(fraction=[0.6])]
)
model.fractional_flow.add(
    Node(13, Point(3.0, -1.0)),
    [fractional_flow.Static(fraction=[0.1])],
)

Setup pump:


In [ ]:
model.pump.add(Node(7, Point(4.0, 1.0)), [pump.Static(flow_rate=[0.5 / 3600])])

Setup level boundary:


In [ ]:
model.level_boundary.add(
    Node(11, Point(2.0, 2.0)), [level_boundary.Static(level=[0.5])]
)
model.level_boundary.add(
    Node(17, Point(6.0, 1.0)), [level_boundary.Static(level=[1.5])]
)

Setup flow boundary:


In [ ]:
model.flow_boundary.add(
    Node(15, Point(3.0, 3.0)), [flow_boundary.Static(flow_rate=[1e-4])]
)
model.flow_boundary.add(
    Node(16, Point(0.0, 1.0)), [flow_boundary.Static(flow_rate=[1e-4])]
)

Setup terminal:


In [ ]:
model.terminal.add(Node(14, Point(3.0, -2.0)))

Setup the edges:


In [ ]:
model.edge.add(model.basin[1], model.manning_resistance[2], "flow")
model.edge.add(model.manning_resistance[2], model.basin[3], "flow")
model.edge.add(model.basin[3], model.tabulated_rating_curve[4], "flow")
model.edge.add(model.tabulated_rating_curve[4], model.fractional_flow[5], "flow")
model.edge.add(model.tabulated_rating_curve[4], model.fractional_flow[8], "flow")
model.edge.add(model.fractional_flow[5], model.basin[6], "flow")
model.edge.add(model.basin[6], model.pump[7], "flow")
model.edge.add(model.fractional_flow[8], model.basin[9], "flow")
model.edge.add(model.pump[7], model.basin[9], "flow")
model.edge.add(model.basin[9], model.linear_resistance[10], "flow")
model.edge.add(model.level_boundary[11], model.linear_resistance[12], "flow")
model.edge.add(model.linear_resistance[12], model.basin[3], "flow")
model.edge.add(model.tabulated_rating_curve[4], model.fractional_flow[13], "flow")
model.edge.add(model.fractional_flow[13], model.terminal[14], "flow")
model.edge.add(model.flow_boundary[15], model.basin[6], "flow")
model.edge.add(model.flow_boundary[16], model.basin[1], "flow")
model.edge.add(model.linear_resistance[10], model.level_boundary[17], "flow")

Let's take a look at the model:


In [ ]:
model.plot()

Write the model to a TOML and GeoPackage:


In [ ]:
datadir = Path("data")
model.write(datadir / "basic/ribasim.toml")

In [ ]:
# | include: false
from subprocess import run

result = run(
    [
        "julia",
        "--project=../../core",
        "--eval",
        f'using Ribasim; Ribasim.main("{datadir.as_posix()}/basic/ribasim.toml")',
    ],
    check=True,
)

Now run the model with `ribasim basic/ribasim.toml`.
After running the model, read back the results:


In [ ]:
df_basin = pd.read_feather(datadir / "basic/results/basin.arrow")
df_basin_wide = df_basin.pivot_table(
    index="time", columns="node_id", values=["storage", "level"]
)
df_basin_wide["level"].plot()

In [ ]:
df_flow = pd.read_feather(datadir / "basic/results/flow.arrow")
df_flow["edge"] = list(zip(df_flow.from_node_id, df_flow.to_node_id))
df_flow["flow_m3d"] = df_flow.flow_rate * 86400
ax = df_flow.pivot_table(index="time", columns="edge", values="flow_m3d").plot()
ax.legend(bbox_to_anchor=(1.3, 1), title="Edge")

# Model with discrete control

The model constructed below consists of a single basin which slowly drains trough a `TabulatedRatingCurve`, but is held within a range around a target level (setpoint) by two connected pumps. These two pumps behave like a reversible pump. When pumping can be done in only one direction, and the other direction is only possible under gravity, use an Outlet for that direction.


Setup the basins:


In [ ]:
model = Model(starttime="2020-01-01 00:00:00", endtime="2021-01-01 00:00:00")

In [ ]:
model.basin.add(
    Node(1, Point(0.0, 0.0)),
    [
        basin.Profile(area=[1000.0, 1000.0], level=[0.0, 1.0]),
        basin.State(level=[20.0]),
    ],
)

Setup the discrete control:


In [ ]:
model.discrete_control.add(
    Node(7, Point(1.0, 0.0)),
    [
        discrete_control.Condition(
            listen_node_id=[1, 1, 1],
            listen_node_type=["Basin", "Basin", "Basin"],
            variable=["level", "level", "level"],
            greater_than=[5.0, 10.0, 15.0],
        ),
        discrete_control.Logic(
            truth_state=["FFF", "U**", "T*F", "**D", "TTT"],
            control_state=["in", "in", "none", "out", "out"],
        ),
    ],
)

The above control logic can be summarized as follows:

- If the level gets above the maximum, activate the control state "out" until the setpoint is reached;
- If the level gets below the minimum, active the control state "in" until the setpoint is reached;
- Otherwise activate the control state "none".


Setup the pump:


In [ ]:
model.pump.add(
    Node(2, Point(1.0, 1.0)),
    [pump.Static(control_state=["none", "in", "out"], flow_rate=[0.0, 2e-3, 0.0])],
)
model.pump.add(
    Node(3, Point(1.0, -1.0)),
    [pump.Static(control_state=["none", "in", "out"], flow_rate=[0.0, 0.0, 2e-3])],
)

The pump data defines the following:

| Control state | Pump #2 flow rate (m/s) | Pump #3 flow rate (m/s) |
| ------------- | ----------------------- | ----------------------- |
| "none"        | 0.0                     | 0.0                     |
| "in"          | 2e-3                    | 0.0                     |
| "out"         | 0.0                     | 2e-3                    |


Setup the level boundary:


In [ ]:
model.level_boundary.add(
    Node(4, Point(2.0, 0.0)), [level_boundary.Static(level=[10.0])]
)

Setup the rating curve:


In [ ]:
model.tabulated_rating_curve.add(
    Node(5, Point(-1.0, 0.0)),
    [tabulated_rating_curve.Static(level=[2.0, 15.0], flow_rate=[0.0, 1e-3])],
)

Setup the terminal:


In [ ]:
model.terminal.add(Node(6, Point(-2.0, 0.0)))

Setup edges:


In [ ]:
model.edge.add(model.basin[1], model.pump[3], "flow")
model.edge.add(model.pump[3], model.level_boundary[4], "flow")
model.edge.add(model.level_boundary[4], model.pump[2], "flow")
model.edge.add(model.pump[2], model.basin[1], "flow")
model.edge.add(model.basin[1], model.tabulated_rating_curve[5], "flow")
model.edge.add(model.tabulated_rating_curve[5], model.terminal[6], "flow")
model.edge.add(model.discrete_control[7], model.pump[2], "control")
model.edge.add(model.discrete_control[7], model.pump[3], "control")

Let’s take a look at the model:


In [ ]:
model.plot()

Listen edges are plotted with a dashed line since they are not present in the "Edge / static" schema but only in the "Control / condition" schema.


In [ ]:
datadir = Path("data")
model.write(datadir / "level_setpoint_with_minmax/ribasim.toml")

In [ ]:
# | include: false
from subprocess import run

run(
    [
        "julia",
        "--project=../../core",
        "--eval",
        f'using Ribasim; Ribasim.main("{datadir.as_posix()}/level_setpoint_with_minmax/ribasim.toml")',
    ],
    check=True,
)

Now run the model with `level_setpoint_with_minmax/ribasim.toml`.
After running the model, read back the results:


In [ ]:
from matplotlib.dates import date2num

df_basin = pd.read_feather(datadir / "level_setpoint_with_minmax/results/basin.arrow")
df_basin_wide = df_basin.pivot_table(
    index="time", columns="node_id", values=["storage", "level"]
)

ax = df_basin_wide["level"].plot()

greater_than = model.discrete_control.condition.df.greater_than

ax.hlines(
    greater_than,
    df_basin.time[0],
    df_basin.time.max(),
    lw=1,
    ls="--",
    color="k",
)

df_control = pd.read_feather(
    datadir / "level_setpoint_with_minmax/results/control.arrow"
)

y_min, y_max = ax.get_ybound()
ax.fill_between(df_control.time[:2], 2 * [y_min], 2 * [y_max], alpha=0.2, color="C0")
ax.fill_between(df_control.time[2:4], 2 * [y_min], 2 * [y_max], alpha=0.2, color="C0")

ax.set_xticks(
    date2num(df_control.time).tolist(),
    df_control.control_state.tolist(),
    rotation=50,
)

ax.set_yticks(greater_than, ["min", "setpoint", "max"])
ax.set_ylabel("level")
plt.show()

The highlighted regions show where a pump is active.


# Model with PID control


Set up the model:


In [ ]:
model = Model(
    starttime="2020-01-01 00:00:00",
    endtime="2020-12-01 00:00:00",
)

Setup the basins:


In [ ]:
model.basin.add(
    Node(2, Point(1.0, 0.0)),
    [basin.Profile(area=[1000.0, 1000.0], level=[0.0, 1.0]), basin.State(level=[6.0])],
)

Setup the pump:


In [ ]:
model.pump.add(
    Node(3, Point(2.0, 0.5)),
    [pump.Static(flow_rate=[0.0])],  # Will be overwritten by PID controller
)

Setup the outlet:


In [ ]:
model.outlet.add(
    Node(6, Point(2.0, -0.5)),
    [outlet.Static(flow_rate=[0.0])],  # Will be overwritten by PID controller
)

Setup flow boundary:


In [ ]:
model.flow_boundary.add(
    Node(1, Point(0.0, 0.0)),
    [flow_boundary.Static(flow_rate=[1e-3])],
)

Setup flow boundary:


In [ ]:
model.level_boundary.add(
    Node(4, Point(3.0, 0.0)),
    [level_boundary.Static(level=[1])],
)

Setup PID control:


In [ ]:
model.pid_control.add(
    Node(5, Point(1.5, 1.0)),
    [
        pid_control.Time(
            time=[
                "2020-01-01 00:00:00",
                "2020-05-01 00:00:00",
                "2020-07-01 00:00:00",
                "2020-12-01 00:00:00",
            ],
            listen_node_id=[2, 2, 2, 2],
            listen_node_type=["Basin", "Basin", "Basin", "Basin"],
            target=[5.0, 5.0, 7.5, 7.5],
            proportional=[-1e-3, 1e-3, 1e-3, 1e-3],
            integral=[-1e-7, 1e-7, -1e-7, 1e-7],
            derivative=[0.0, 0.0, 0.0, 0.0],
        )
    ],
)
model.pid_control.add(
    Node(7, Point(1.5, -1.0)),
    [
        pid_control.Time(
            time=[
                "2020-01-01 00:00:00",
                "2020-05-01 00:00:00",
                "2020-07-01 00:00:00",
                "2020-12-01 00:00:00",
            ],
            listen_node_id=[2, 2, 2, 2],
            listen_node_type=["Basin", "Basin", "Basin", "Basin"],
            target=[5.0, 5.0, 7.5, 7.5],
            proportional=[-1e-3, 1e-3, 1e-3, 1e-3],
            integral=[-1e-7, 1e-7, -1e-7, 1e-7],
            derivative=[0.0, 0.0, 0.0, 0.0],
        )
    ],
)

Note that the coefficients for the pump and the outlet are equal in magnitude but opposite in sign. This way the pump and the outlet equally work towards the same goal, while having opposite effects on the controlled basin due to their connectivity to this basin.


Setup the edges:


In [ ]:
model.edge.add(model.flow_boundary[1], model.basin[2], "flow")
model.edge.add(model.basin[2], model.pump[3], "flow")
model.edge.add(model.pump[3], model.level_boundary[4], "flow")
model.edge.add(model.level_boundary[4], model.outlet[6], "flow")
model.edge.add(model.outlet[6], model.basin[2], "flow")
model.edge.add(model.pid_control[5], model.pump[3], "control")
model.edge.add(model.pid_control[7], model.outlet[6], "control")

Let's take a look at the model:


In [ ]:
model.plot()

Write the model to a TOML and GeoPackage:


In [ ]:
datadir = Path("data")
model.write(datadir / "pid_control/ribasim.toml")

In [ ]:
# | include: false
from subprocess import run

run(
    [
        "julia",
        "--project=../../core",
        "--eval",
        f'using Ribasim; Ribasim.main("{datadir.as_posix()}/pid_control/ribasim.toml")',
    ],
    check=True,
)

Now run the model with `ribasim pid_control/ribasim.toml`.
After running the model, read back the results:


In [ ]:
from matplotlib.dates import date2num

df_basin = pd.read_feather(datadir / "pid_control/results/basin.arrow")
df_basin_wide = df_basin.pivot_table(
    index="time", columns="node_id", values=["storage", "level"]
)
ax = df_basin_wide["level"].plot()
ax.set_ylabel("level [m]")

# Plot target level
level_demands = model.pid_control.time.df.target.to_numpy()[:4]
times = date2num(model.pid_control.time.df.time)[:4]
ax.plot(times, level_demands, color="k", ls=":", label="target level")
pass

# Model with allocation (user demand)


Setup a model:


In [ ]:
model = Model(
    starttime="2020-01-01 00:00:00",
    endtime="2020-01-20 00:00:00",
)

Setup the basins:


In [ ]:
basin_data = [
    basin.Profile(area=[300_000.0, 300_000.0], level=[0.0, 1.0]),
    basin.State(level=[1.0]),
]

model.basin.add(
    Node(2, Point(1.0, 0.0), subnetwork_id=1),
    basin_data,
)
model.basin.add(
    Node(5, Point(3.0, 0.0), subnetwork_id=1),
    basin_data,
)
model.basin.add(
    Node(12, Point(4.5, 1.0), subnetwork_id=1),
    basin_data,
)

Setup the flow boundary:


In [ ]:
model.flow_boundary.add(
    Node(1, Point(0.0, 0.0), subnetwork_id=1), [flow_boundary.Static(flow_rate=[2.0])]
)

Setup the linear resistance:


In [ ]:
model.linear_resistance.add(
    Node(4, Point(2.0, 0.0), subnetwork_id=1),
    [linear_resistance.Static(resistance=[0.06])],
)

Setup the tabulated rating curve:


In [ ]:
model.tabulated_rating_curve.add(
    Node(7, Point(4.0, 0.0), subnetwork_id=1),
    [tabulated_rating_curve.Static(level=[0.0, 0.5, 1.0], flow_rate=[0.0, 0.0, 2.0])],
)

Setup the fractional flow:


In [ ]:
model.fractional_flow.add(
    Node(8, Point(4.5, 0.0), subnetwork_id=1),
    [fractional_flow.Static(fraction=[0.6, 0.9], control_state=["divert", "close"])],
)
model.fractional_flow.add(
    Node(9, Point(4.5, 0.5), subnetwork_id=1),
    [fractional_flow.Static(fraction=[0.4, 0.1], control_state=["divert", "close"])],
)

Setup the terminal:


In [ ]:
model.terminal.add(Node(10, Point(5.0, 0.0), subnetwork_id=1))

Setup the discrete control:


In [ ]:
model.discrete_control.add(
    Node(11, Point(4.5, 0.25), subnetwork_id=1),
    [
        discrete_control.Condition(
            listen_node_id=[5],
            listen_node_type=["Basin"],
            variable=["level"],
            greater_than=[0.52],
        ),
        discrete_control.Logic(
            truth_state=["T", "F"], control_state=["divert", "close"]
        ),
    ],
)

Setup the users:


In [ ]:
model.user_demand.add(
    Node(6, Point(3.0, 1.0), subnetwork_id=1),
    [
        user_demand.Static(
            demand=[1.5], return_factor=[0.0], min_level=[-1.0], priority=[1]
        )
    ],
)
model.user_demand.add(
    Node(13, Point(5.0, 1.0), subnetwork_id=1),
    [
        user_demand.Static(
            demand=[1.0], return_factor=[0.0], min_level=[-1.0], priority=[3]
        )
    ],
)
model.user_demand.add(
    Node(3, Point(1.0, 1.0), subnetwork_id=1),
    [
        user_demand.Time(
            demand=[0.0, 1.0, 1.2, 1.2],
            return_factor=[0.0, 0.0, 0.0, 0.0],
            min_level=[-1.0, -1.0, -1.0, -1.0],
            priority=[1, 1, 2, 2],
            time=2 * ["2020-01-01 00:00:00", "2020-01-20 00:00:00"],
        )
    ],
)

Setup the allocation:


In [ ]:
model.allocation = ribasim.Allocation(use_allocation=True, timestep=86400)

Setup the edges:

In [ ]:
model.edge.add(model.flow_boundary[1], model.basin[2], "flow", subnetwork_id=1)
model.edge.add(model.basin[2], model.user_demand[3], "flow")
model.edge.add(model.basin[2], model.linear_resistance[4], "flow")
model.edge.add(model.linear_resistance[4], model.basin[5], "flow")
model.edge.add(model.basin[5], model.user_demand[6], "flow")
model.edge.add(model.basin[5], model.tabulated_rating_curve[7], "flow")
model.edge.add(model.tabulated_rating_curve[7], model.fractional_flow[8], "flow")
model.edge.add(model.user_demand[3], model.basin[2], "flow")
model.edge.add(model.user_demand[6], model.basin[5], "flow")
model.edge.add(model.tabulated_rating_curve[7], model.fractional_flow[9], "flow")
model.edge.add(model.fractional_flow[8], model.terminal[10], "flow")
model.edge.add(model.fractional_flow[9], model.basin[12], "flow")
model.edge.add(model.basin[12], model.user_demand[13], "flow")
model.edge.add(model.user_demand[13], model.terminal[10], "flow")
model.edge.add(model.discrete_control[11], model.fractional_flow[8], "control")
model.edge.add(model.discrete_control[11], model.fractional_flow[9], "control")

Let's take a look at the model:


In [ ]:
model.plot()

Write the model to a TOML and GeoPackage:


In [ ]:
datadir = Path("data")
model.write(datadir / "allocation_example/ribasim.toml")

In [ ]:
# | include: false
from subprocess import run

run(
    [
        "julia",
        "--project=../../core",
        "--eval",
        f'using Ribasim; Ribasim.main("{datadir.as_posix()}/allocation_example/ribasim.toml")',
    ],
    check=True,
)

Now run the model with `ribasim allocation_example/ribasim.toml`.
After running the model, read back the results:


In [ ]:
import matplotlib.ticker as plticker

df_allocation = pd.read_feather(datadir / "allocation_example/results/allocation.arrow")
df_allocation_wide = df_allocation.pivot_table(
    index="time",
    columns=["node_type", "node_id", "priority"],
    values=["demand", "allocated", "realized"],
)
df_allocation_wide = df_allocation_wide.loc[:, (df_allocation_wide != 0).any(axis=0)]

fig, axs = plt.subplots(1, 3, figsize=(8, 5))
fig.suptitle(f"Objective type: {model.allocation.objective_type}", fontweight="bold")

df_allocation_wide["demand"].plot(ax=axs[0], ls=":")
df_allocation_wide["allocated"].plot(ax=axs[1], ls="--")
df_allocation_wide["realized"].plot(ax=axs[2])

fig.tight_layout()
loc = plticker.MultipleLocator(2)

axs[0].set_ylabel("level [m]")

for ax, title in zip(axs, ["Demand", "Allocated", "Abstracted"]):
    ax.set_title(title)
    ax.set_ylim(0.0, 1.6)
    ax.xaxis.set_major_locator(loc)

Some things to note about this plot:

- Abstraction behaves somewhat erratically at the start of the simulation. This is because allocation is based on flows computed in the physical layer, and at the start of the simulation these are not known yet.

- Although there is a plotted line for abstraction per priority, abstraction is actually accumulated over all priorities per user.


In [ ]:
df_basin = pd.read_feather(datadir / "allocation_example/results/basin.arrow")
df_basin_wide = df_basin.pivot_table(
    index="time", columns="node_id", values=["storage", "level"]
)

ax = df_basin_wide["level"].plot()
ax.set_title("Basin levels")
ax.set_ylabel("level [m]")

# Model with allocation (basin supply/demand)


Setup a model:


In [ ]:
model = ribasim.Model(
    starttime="2020-01-01 00:00:00",
    endtime="2020-02-01 00:00:00",
)

Setup the basins:


In [ ]:
basin_data = [
    basin.Profile(area=[1e3, 1e3], level=[0.0, 1.0]),
    basin.State(level=[0.5]),
]
model.basin.add(
    Node(2, Point(1.0, 0.0)),
    [
        *basin_data,
        basin.Time(
            time=["2020-01-01 00:00:00", "2020-01-16 00:00:00"],
            drainage=[0.0, 0.0],
            potential_evaporation=[0.0, 0.0],
            infiltration=[0.0, 0.0],
            precipitation=[1e-6, 0.0],
            urban_runoff=[0.0, 0.0],
        ),
    ],
)
model.basin.add(
    Node(5, Point(2.0, -1.0)),
    [
        *basin_data,
        basin.Static(
            drainage=[0.0],
            potential_evaporation=[0.0],
            infiltration=[0.0],
            precipitation=[0.0],
            urban_runoff=[0.0],
        ),
    ],
)
profile = pd.DataFrame(
    data={"node_id": [2, 2, 5, 5], "area": 1e3, "level": [0.0, 1.0, 0.0, 1.0]}
)

Setup the flow boundary:


In [ ]:
model.flow_boundary.add(
    Node(1, Point(0.0, 0.0)), [flow_boundary.Static(flow_rate=[1e-3])]
)

Setup allocation level control:


In [ ]:
model.level_demand.add(
    Node(4, Point(1.0, -1.0)),
    [level_demand.Static(priority=[1], min_level=[1.0], max_level=[1.5])],
)

Setup the users:


In [ ]:
model.user_demand.add(
    Node(3, Point(2.0, 0.0)),
    [
        user_demand.Static(
            priority=[2], demand=[1.5e-3], return_factor=[0.2], min_level=[0.2]
        )
    ],
)

Setup the allocation:


In [ ]:
model.allocation = ribasim.Allocation(use_allocation=True, timestep=1e5)

Setup the edges:


In [ ]:
model.edge.add(model.flow_boundary[1], model.basin[2], "flow", subnetwork_id=2)
model.edge.add(model.basin[2], model.user_demand[3], "flow")
model.edge.add(model.level_demand[4], model.basin[2], "control")
model.edge.add(model.user_demand[3], model.basin[5], "flow")
model.edge.add(model.level_demand[4], model.basin[5], "control")

Let's take a look at the model:


In [ ]:
model.plot()

Write the model to a TOML and GeoPackage:


In [ ]:
model.write(datadir / "level_demand/ribasim.toml")

In [ ]:
# | include: false
from subprocess import run

run(
    [
        "julia",
        "--project=../../core",
        "--eval",
        f'using Ribasim; Ribasim.main("{datadir.as_posix()}/level_demand/ribasim.toml")',
    ],
    check=True,
)

Now run the model with `ribasim level_demand/ribasim.toml`.
After running the model, read back the results:


In [ ]:
df_basin = pd.read_feather(datadir / "level_demand/results/basin.arrow")
df_basin = df_basin[df_basin.node_id == 2]
df_basin_wide = df_basin.pivot_table(
    index="time", columns="node_id", values=["storage", "level"]
)
ax = df_basin_wide["level"].plot()
where_allocation = (
    df_basin_wide.index - df_basin_wide.index[0]
).total_seconds() % model.allocation.timestep == 0
where_allocation[0] = False
df_basin_wide[where_allocation]["level"].plot(
    style="o",
    ax=ax,
)
ax.set_ylabel("level [m]")

In the plot above, the line denotes the level of Basin #2 over time and the dots denote the times at which allocation optimization was run, with intervals of $\Delta t_{\text{alloc}}$.
The Basin level is a piecewise linear function of time, with several stages explained below.

Constants:

- $d$: UserDemand #3 demand,
- $\phi$: Basin #2 precipitation rate,
- $q$: LevelBoundary flow.


Stages:

- In the first stage the UserDemand abstracts fully, so the net change of Basin #2 is $q + \phi - d$;
- In the second stage the Basin takes precedence so the UserDemand doesn't abstract, hence the net change of Basin #2 is $q + \phi$;
- In the third stage (and following stages) the Basin no longer has a positive demand, since precipitation provides enough water to get the Basin to its target level. The FlowBoundary flow gets fully allocated to the UserDemand, hence the net change of Basin #2 is $\phi$;
- In the fourth stage the Basin enters its surplus stage, even though initially the level is below the maximum level. This is because the simulation anticipates that the current precipitation is going to bring the Basin level over its maximum level. The net change of Basin #2 is now $q + \phi - d$;
- At the start of the fifth stage the precipitation stops, and so the UserDemand partly uses surplus water from the Basin to fulfill its demand. The net change of Basin #2 becomes $q - d$.
- In the final stage the Basin is in a dynamical equilibrium, since the Basin has no supply so the user abstracts precisely the flow from the LevelBoundary.
